In [1]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
import MeCab
import numpy as np

In [2]:
def readCSV(file_pass, name, file_type):
    df = pd.read_csv(file_pass+name+file_type, dtype={'label': 'str'}, encoding="utf-8")
    return df

def sep(body):
    mt = MeCab.Tagger('mecabrc')
    mt.parse("")
    node = mt.parseToNode(str(body))
    return getNouns(node)

# 名詞のみを抽出
def checkNoun(noun):
    zyogai = ['副詞可能', "非自立", "代名詞","助詞類接続"]
    if(noun[0] == '名詞'):
        if(noun[1] in zyogai):
            return False
        else:
            return True
    else:
        return False
    
def getNouns(node):
    words = []
    word_stack = ""
    while node:
        noun = node.feature.split(",")
        word = node.surface
        if(checkNoun(noun)):
            word_stack += word
        if(word_stack != "" and noun[0] != '名詞'):
            if(checkNum(word_stack)):
                words.append(word_stack)
            word_stack = ""
        node = node.next
    words = " ".join(words)
    return words


# 12月 1.2%などの数字関連を省く
def checkNum(word_stack):
    mt = MeCab.Tagger()
    mt.parse("")

    node = mt.parseToNode(word_stack)
    while node:
        noun = node.feature.split(",")
        if(noun[1] == '数'):
            return False
        node = node.next

    return True


In [4]:
file_pass = "./"
#3が自作関数での分割
file_name = "reuters_article"
file_type = ".csv"
dict_list = []
# print(dict_list)
df = readCSV(file_pass, file_name, file_type)
# print(df["label"])
body = []
label_train = []

In [5]:
body = [sep(row) for row in df["body"]]

In [16]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=10000)
tf = tf_vectorizer.fit_transform(body)
tf_feature_names = tf_vectorizer.get_feature_names()

lda = LatentDirichletAllocation(n_components=20, max_iter=5, learning_method='online', learning_offset=50, random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=20, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [19]:
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([tf_feature_names[i] for i in topic.argsort()[: -21: -1]]))

Topic 0:
日経平均 東京株式市場 上昇 市場 上値 先行 前場 買い 反発 寄り付き 堅調 国内証券 輸出株 好感 反落 観測 銀行株 軟調 中心 円安
Topic 1:
赤字 売上高 予想 見通し 利益 発表 同社 黒字 影響 営業利益 計画 好調 社長 業績 減少 下方修正 トムソンロイターエスティメーツ 増加 想定 ロイターエスティメーツ
Topic 2:
発表 rbs de 社債 pa 前週 ubs vw 入札 vx スイス 拡充 増加 供給 hsbc スウェーデン ポルシェ 明らか 調査 英政府
Topic 3:
gm クライスラー 明らか 発表 販売 同社 トヨタ 計画 生産 工場 可能性 米ゼネラルモーターズ 方針 発売 自動車メーカー 提携 ceo 販売台数 オペル ゼネラルモーターズ
Topic 4:
合意 協議 表明 問題 明らか 首相 開催 イラン 提案 会談 会合 予定 必要 要請 主張 批判 協力 決定 オバマ米大統領 新た
Topic 5:
中国 北朝鮮 米国 韓国 可能性 明らか 発表 確認 同国 人民元 感染 ロシア 中国政府 当局者 期限 中国経済 発射 日本 核実験 報道官
Topic 6:
政府 民主党 考え 対応 選挙 必要 議論 会見 指摘 日本 強調 政策 政権 認識 国民 財源 予算 自民党 対策 検討
Topic 7:
減少 発表 予想 生産 上昇 前月 低下 gdp ロイター 増加 水準 大幅 伸び 輸出 改善 悪化 最大 調査 影響 製造業
Topic 8:
ドル 上昇 ユーロ 下落 高値 取引 原油価格 更新 豪ドル 背景 水準 通貨 安値 米ドル ドル安 原油 米原油先物 opec ポンド 対ユーロ
Topic 9:
ecb 日本 欧州中央銀行 ドイツ 投資 米国 eu ユーロ圏 世界 欧州 インド 英国 必要 インタビュー ロシア トリシェ総裁 欧州連合 目標 設定 金融サミット
Topic 10:
金融機関 銀行 損失 可能性 シティ 問題 公的資金 カメ 発表 明らか シティグループ 必要 資本 計上 注入 指摘 bac 破たん 市場 jpm
Topic 11:
オバマ大統領 大統領 オバマ氏 民主党 オバマ政権 勝利 支持 共和党 指名 米国 獲得 投票 調査 訂正 演説 支持率 マケイン氏 大